<a href="https://colab.research.google.com/github/MengOonLee/BertelsmannAITrack/blob/Lesson10/AnnotationsPyTorchAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Annotations

In [1]:
import torch
print(torch.__version__)

class MyModule(torch.jit.ScriptModule):
  def __init__(self, N, M):
    super(MyModule, self).__init__()
    self.weight = torch.nn.Parameter(torch.rand(N, M))

  @torch.jit.script_method
  def forward(self, input):
    if input.sum() > 0:
      output = self.weight.mv(input)
    else:
      output = self.weight + input
    return output

my_module = MyModule(10, 20)
sm = torch.jit.script(my_module)

1.3.1


# PyTorch C++ API

In [2]:
# Download LibTorch distribution
!wget -q -O libtorch-shared-with-deps-1.3.1.zip https://download.pytorch.org/libtorch/cu101/libtorch-shared-with-deps-1.3.1.zip && unzip libtorch-shared-with-deps-1.3.1.zip && rm -rf libtorch-shared-with-deps-1.3.1.zip

!wget -q -O libtorch-cxx11-abi-shared-with-deps-1.3.1.zip https://download.pytorch.org/libtorch/cu101/libtorch-cxx11-abi-shared-with-deps-1.3.1.zip && unzip libtorch-cxx11-abi-shared-with-deps-1.3.1.zip && rm -rf libtorch-cxx11-abi-shared-with-deps-1.3.1.zip

Archive:  libtorch-shared-with-deps-1.3.1.zip
   creating: libtorch/
   creating: libtorch/lib/
  inflating: libtorch/lib/libpytorch_qnnpack.a  
  inflating: libtorch/lib/libc10.so  
  inflating: libtorch/lib/libgloo.a  
  inflating: libtorch/lib/libcaffe2_nvrtc.so  
  inflating: libtorch/lib/libc10d.a  
  inflating: libtorch/lib/libgmock_main.a  
  inflating: libtorch/lib/libmkldnn.a  
  inflating: libtorch/lib/libcaffe2_protos.a  
  inflating: libtorch/lib/libCaffe2_perfkernels_avx2.a  
  inflating: libtorch/lib/libc10d_cuda_test.so  
  inflating: libtorch/lib/libnnpack.a  
  inflating: libtorch/lib/libnnpack_reference_layers.a  
  inflating: libtorch/lib/libbenchmark.a  
  inflating: libtorch/lib/libcpuinfo.a  
  inflating: libtorch/lib/libonnx.a  
  inflating: libtorch/lib/libprotoc.a  
  inflating: libtorch/lib/libfbgemm.a  
  inflating: libtorch/lib/libpthreadpool.a  
  inflating: libtorch/lib/libgtest_main.a  
  inflating: libtorch/lib/libonnx_proto.a  
  inflating: libtorch/lib

In [0]:
%%bash

# Make a simple app directory
mkdir example-app

In [4]:
%%file example-app/example-app.cpp

#include <torch/script.h> // One-stop header.

#include <iostream>
#include <memory>

int main(int argc, const char* argv[]) {
  if (argc != 2) {
    std::cerr << "usage: example-app <path-to-exported-script-module>\n";
    return -1;
  }


  torch::jit::script::Module module;
  try {
    // Deserialize the ScriptModule from a file using torch::jit::load().
    module = torch::jit::load(argv[1]);
  }
  catch (const c10::Error& e) {
    std::cerr << "error loading the model\n";
    return -1;
  }

  std::cout << "ok\n";
}

Writing example-app/example-app.cpp


In [5]:
%%file example-app/CMakeLists.txt

cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(custom_ops)

find_package(Torch REQUIRED)

add_executable(example-app example-app.cpp)
target_link_libraries(example-app "${TORCH_LIBRARIES}")
set_property(TARGET example-app PROPERTY CXX_STANDARD 11)

Writing example-app/CMakeLists.txt


In [10]:
%%bash

cd example-app/
mkdir build
cd build
cmake -DCMAKE_PREFIX_PATH=/content/libtorch ..
cmake --build . --config Release

-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "10.0") 
-- Caffe2: CUDA detected: 10.0
-- Caffe2: CUDA nvcc is: /usr/local/cuda/bin/nvcc
-- Caffe2: CUDA toolkit directory: /usr/local/cuda
-- Caffe2: Header version is: 10.0
-- Found CUDNN: /usr/lib/x86_64-linux-gnu/libcudnn.so  
-- Found cuDNN: v7.6.5  (include: /usr/include, library: /usr/lib/x86_64-linux-gnu/libcudnn.so)
-- Autodetected CUDA architecture(s):  6.0
-- Added CUDA NVCC flags for: -gencode;arch=compute_60,code=sm_60
-- Found torch: /content/libtorch/lib/libtorch.so  
-- Configuring done
-- Generating done
-- Build files have been written to: /content/example-app/build
Scanning dependencies

mkdir: cannot create directory ‘build’: File exists


In [0]:
import torch
import torchvision

# An instance of your model
model = torchvision.models.resnet18()
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

traced_script_module.save("traced_resnet_model.pt")

In [14]:
%%bash

/example-app/build# ./example-app traced_resnet_model.pt

bash: line 2: /example-app/build#: No such file or directory
